In [1]:
import pandas as pd
import datetime as dt
import numpy as np
%config IPCompleter.greedy=True

### A few ideas that come to mind:
* Change the Date Opened column entries to datetime objects
* Make the entries of the recall number integers, be smart about how to come up with unique integers
* Make the entries of the Pounds Recalled column integers
* Make the entries of the Recall Class column more descriptive String objects
* Make a table where I have the recall reasons all as one df and by year

## I. Load the data

In [2]:
df_dict = {}
root = 'datasets'
filenames = ['recalls_2005.csv', 'recalls_2006.csv', 'recalls_2007.csv', 'recalls_2008.csv', 'recalls_2009.csv', 'recalls_2010.csv', 'recalls_2011.csv', 'recalls_2012.csv', 'recalls_2013.csv', 'recalls_2014.csv', 'recalls_2015.csv', 'recalls_2016.csv', 'recalls_2017.csv', 'recalls_2018.csv']

# Define a function to load all the files into dataframes and place them into a dictionary: the key is the year and the value the data as a pandas dataframe
def load_data(filenames):
    for filename in filenames:
        key = filename.split('.')[0].split('_')[1]
        df_dict[key] = pd.read_csv(root + '/' + filename)

In [3]:
load_data(filenames)

In [4]:
df_dict['2016'].head(8)

,Recall Number,Open Date,Class,Pounds Recalled,Product,Problem Type
0,001-2016,4-Jan-16,2,"89,568",Beef products,Extraneous Material
1,002-2016,5-Jan-16,1,14,Cajun Hickory Smoked Pork Tasso,Listeria monocytogenes
2,003-2016,5-Jan-16,1,"1,125",Chicken products,Other
3,004-2016,6-Jan-16,1,"7,687","Beef, Pork, and Chicken Products",Other
4,005-2016,8-Jan-16,2,"4,040",Pork Sausage,Undeclared Substance
5,006-2016,16-Jan-16,2,"22,182",Sweet Apple Chicken Sausage,Undeclared Substance
6,007-2016,20-Jan-16,1,410,Ground Beef,E. coli
7,008-2016,22-Jan-16,3,"549,539",Chili without Beans,Undeclared Substance


In [5]:
df_dict['2016'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 6 columns):
Recall Number      122 non-null object
Open Date          122 non-null object
Class              122 non-null int64
Pounds Recalled    122 non-null object
Product            122 non-null object
Problem Type       122 non-null object
dtypes: int64(1), object(5)
memory usage: 5.8+ KB


Let's take a peek at a sample content of the data dictionary just created.

In [6]:
# Get the second item in the dictionary
recalls_ls = list(df_dict.items())
# Get a random index of the items in the dictionary
idx = np.random.randint(len(recalls_ls))
# Get one of the recalls data in the dictionary
year, df = recalls_ls[idx]
num = 5
print('\n\nThese are the first {} rows of the recalls data of {}:\n'.format(num, year))
df.head(num)



These are the first 5 rows of the recalls data of 2015:



,Recall Number,Date Opened,Recall Class,Pounds Recalled,Product,Problem Type
0,001-2015,1/2/2015,1,"35,275",Australian Bone-In Lamb Shoulder,Other
1,002-2015,1/7/2015,1,359,Boneless Beef,Other
2,003-2015,1/7/2015,1,"35,865",Chili With Beans,Undeclared Allergen
3,004-2015,1/8/2015,1,"48,139",RTE Beef and Pork Products,Extraneous Material
4,005-2015,1/8/2015,1,"7,092",Halal Beef Soujouk Dried Beef Sausage,Undeclared Allergen


The data was correctly loaded and correctly indexed in the dictionary.

## II. Investigate the data

### 1. Investigate column names and positions

As we were taking a peek at the data in loaded we could see some inconsistencies between column names. Let's investigate this a bit further.

In [7]:
def extract_col_names_groups(df_dict):
    col_names_dict = {}
    for year, df in df_dict.items():
        columns = tuple(list(df.columns))
        if columns in col_names_dict:
            col_names_dict[columns].append(year)
        else:
            col_names_dict[columns] = [year]
    return col_names_dict

In [8]:
col_names_dict = extract_col_names_groups(df_dict)

In [9]:
col_names_dict

{('Date Opened',
  'Recall Number',
  'Recall Class',
  'Product',
  'Reason for Recall',
  'Pounds Recalled'): ['2005', '2006', '2007', '2008', '2009'],
 ('Recall Date',
  'Recall Number',
  'Recall Class',
  'Product',
  'Reason for Recall',
  'Pounds Recalled'): ['2010', '2011', '2012', '2013', '2014'],
 ('Recall Number',
  'Date Opened',
  'Recall Class',
  'Pounds Recalled',
  'Product',
  'Problem Type'): ['2015'],
 ('Recall Number',
  'Open Date',
  'Class',
  'Pounds Recalled',
  'Product',
  'Problem Type'): ['2016', '2017', '2018']}

There appears to be 4 groups in the dataframes when it comes to column naming. The first group with uniform names and position across the columns are the dataframes for the year 2005 through 2009.
The second group covers years 2010 to 2014. 
The third group is the year 2015.
And the fourth group goes from 2015 to 2018.

Next, let's check if there is indeed uniformity within those groups.

In [10]:
grouped_cols = {}
for idx, (cols, years) in enumerate(col_names_dict.items()):
    grouped_cols[idx+1] = years
grouped_cols

{1: ['2005', '2006', '2007', '2008', '2009'],
 2: ['2010', '2011', '2012', '2013', '2014'],
 3: ['2015'],
 4: ['2016', '2017', '2018']}

In [11]:
rdi = np.random.randint
year_g1, year_g2, year_g3, year_g4 = rdi(2005, 2010), rdi(2010, 2015), 2015, rdi(2016, 2018)
years = str(year_g1), str(year_g2), str(year_g3), str(year_g4)

In [12]:
col_df = pd.DataFrame([df_dict[years[0]].columns, df_dict[years[1]].columns, df_dict[years[2]].columns, df_dict[years[3]].columns], index=['Col Group 1', 'Col Group 2', 'Col Group 3', 'Col Group 4'], columns=['Col ' + str(i) for i in range(6)])

col_df

,Col 0,Col 1,Col 2,Col 3,Col 4,Col 5
Col Group 1,Date Opened,Recall Number,Recall Class,Product,Reason for Recall,Pounds Recalled
Col Group 2,Recall Date,Recall Number,Recall Class,Product,Reason for Recall,Pounds Recalled
Col Group 3,Recall Number,Date Opened,Recall Class,Pounds Recalled,Product,Problem Type
Col Group 4,Recall Number,Open Date,Class,Pounds Recalled,Product,Problem Type


Let's one hot encode the values to get a better idea of what's going on with the column names.

In [13]:
all_cols =  []
for year in years:
    all_cols += [col for col in df_dict[year].columns if col not in all_cols]

In [14]:
rows = []
for year in years:
    year_columns = df_dict[year].columns
    row = [1 if col in year_columns else 0 for col in all_cols ]
    rows.append(row)

In [15]:
cols_group_df = pd.DataFrame(rows, columns=all_cols, index=['Col Group 1', 'Col Group 2', 'Col Group 3', 'Col Group 4'])
cols_group_df

,Date Opened,Recall Number,Recall Class,Product,Reason for Recall,Pounds Recalled,Recall Date,Problem Type,Open Date,Class
Col Group 1,1,1,1,1,1,1,0,0,0,0
Col Group 2,0,1,1,1,1,1,1,0,0,0
Col Group 3,1,1,1,1,0,1,0,1,0,0
Col Group 4,0,1,0,1,0,1,0,1,1,1



For all groups there seems to be a value for Recall Number, Product and Pounds Recalled so we can drop those columns and investigate further the columns with inconsistencies.

In [16]:
cols_group_df = cols_group_df.drop([col for col in cols_group_df.columns if cols_group_df[col].sum()==4], axis=1)

In [17]:
cols_group_df

,Date Opened,Recall Class,Reason for Recall,Recall Date,Problem Type,Open Date,Class
Col Group 1,1,1,1,0,0,0,0
Col Group 2,0,1,1,1,0,0,0
Col Group 3,1,1,0,0,1,0,0
Col Group 4,0,0,0,0,1,1,1


### Remarks

1. There are 3 different names for the column with the date the recall was initiated: Date Opened, Recall Date and Open Date. 
2. There are 2 column names for the class of the recall: Recall Class and Class
3. There are are 2 different column names for the reason the recall was initiated: Reason for Recall and Problem Type.

### Solution

1. The column with the date of the recall will be renamed Recall Date for the dataframes of groups 1, 3 and 4
2. The column with the class of the recall will be renamed Recall Class for the dataframes of group 4
3. The reason for the recall column will be renamed Recall Reason across all the dataframes

In [18]:
col_df

,Col 0,Col 1,Col 2,Col 3,Col 4,Col 5
Col Group 1,Date Opened,Recall Number,Recall Class,Product,Reason for Recall,Pounds Recalled
Col Group 2,Recall Date,Recall Number,Recall Class,Product,Reason for Recall,Pounds Recalled
Col Group 3,Recall Number,Date Opened,Recall Class,Pounds Recalled,Product,Problem Type
Col Group 4,Recall Number,Open Date,Class,Pounds Recalled,Product,Problem Type


### Remarks

1. The column with the class of the recall is always the third column for all dataframes
2. The date the recall was initiated is the 1rst column of the dataframes of groups 1 and 2 but the second column for groups 3 and 4
3. The identifying number of the recall is the 2nd column for the dataframes of groups 1 and 2 but the first column for groups 3 and 4
4. The product column is the 4th column of the dataframes of grous 1 and 2 but the 5th of dataframes of groups 3 and 4
5. The Pounds Recalled column is the 4th column of the dataframes of groups 3 and 4 but the 6th of dataframes of groups 1 and 2
6. The reason for the recall column is the 5th column of the dataframes of groups 1 and 2 but the 6th of dataframes of groups 3 and 4

### Solution

The columns will be reorganized across all dataframes to be in this order: Recall Number, Recall Date, Recall Class, Product, Recall Reason, Pounds Recalled

### 2. Fixing column names and positions

In [19]:
def fix_col_name_and_pos(year, names_changes):
    new_cols = ['Recall Number', 'Recall Date', 'Risk Level', 'Product', 'Recall Reason', 'Pounds Recalled']
    df_dict[year] = df_dict[year].rename(names_changes, axis=1)
    df_dict[year] = df_dict[year].rename({'Recall Class': 'Risk Level'}, axis=1)
    df_dict[year] = df_dict[year][new_cols]

In [20]:
changes_dict = {1: {'Date Opened': 'Recall Date', 'Reason for Recall': 'Recall Reason'},
                2: {'Reason for Recall': 'Recall Reason'},
                3: {'Date Opened': 'Recall Date', 'Problem Type': 'Recall Reason'},
                4: {'Open Date': 'Recall Date', 'Class': 'Recall Class', 'Problem Type': 'Recall Reason'}}

for group, years in grouped_cols.items():
    for year in years:
        fix_col_name_and_pos(year, changes_dict[group])

In [21]:
rdi = np.random.randint
year_g1, year_g2, year_g3, year_g4 = rdi(2005, 2010), rdi(2010, 2015), 2015, rdi(2016, 2018)
years = str(year_g1), str(year_g2), str(year_g3), str(year_g4)

In [22]:
col_df = pd.DataFrame([df_dict[years[0]].columns, df_dict[years[1]].columns, df_dict[years[2]].columns, df_dict[years[3]].columns], index=['Col Group 1', 'Col Group 2', 'Col Group 3', 'Col Group 4'], columns=['Col ' + str(i) for i in range(6)])

col_df

,Col 0,Col 1,Col 2,Col 3,Col 4,Col 5
Col Group 1,Recall Number,Recall Date,Risk Level,Product,Recall Reason,Pounds Recalled
Col Group 2,Recall Number,Recall Date,Risk Level,Product,Recall Reason,Pounds Recalled
Col Group 3,Recall Number,Recall Date,Risk Level,Product,Recall Reason,Pounds Recalled
Col Group 4,Recall Number,Recall Date,Risk Level,Product,Recall Reason,Pounds Recalled


### 3. Investigate and Fix Columns Data Types

The entries of the Pounds Recalled column would be better off being of type integer.

In [23]:
class_names_meaning = {'I':'High', 'II':'Medium', 'III':'Low'}
class_names_meaning_2016 = {1:'High', 2:'Medium', 3:'Low'}

In [24]:
for year, df in df_dict.items():
    if year=='2016':
        df_dict[year]['Risk Level'] = df_dict[year]['Risk Level'].map(class_names_meaning_2016)
    else:
        df_dict[year]['Risk Level'] = df_dict[year]['Risk Level'].map(class_names_meaning)

In [25]:
df_dict['2016'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 6 columns):
Recall Number      122 non-null object
Recall Date        122 non-null object
Risk Level         122 non-null object
Product            122 non-null object
Recall Reason      122 non-null object
Pounds Recalled    122 non-null object
dtypes: object(6)
memory usage: 5.8+ KB


In [26]:
df_dict['2016'].head()

,Recall Number,Recall Date,Risk Level,Product,Recall Reason,Pounds Recalled
0,001-2016,4-Jan-16,Medium,Beef products,Extraneous Material,"89,568"
1,002-2016,5-Jan-16,High,Cajun Hickory Smoked Pork Tasso,Listeria monocytogenes,14
2,003-2016,5-Jan-16,High,Chicken products,Other,"1,125"
3,004-2016,6-Jan-16,High,"Beef, Pork, and Chicken Products",Other,"7,687"
4,005-2016,8-Jan-16,Medium,Pork Sausage,Undeclared Substance,"4,040"


From the two cells below I conclude that I don't need to know the product that was recalled. I, however, am extremely interested in the type of product it is. Beef? Pork? Poultry? This is going to require some substantial investigating to find how to get these into those categories.

In [27]:
df = df_dict['2005']

In [28]:
df['Product'].value_counts()

Sausage Products                               4
Chicken Products                               3
Frozen Ground Beef Patties                     3
Chicken Salad                                  2
Ham Products                                   2
RTE Meat Products                              2
Ground Beef                                    2
Spanish Sausage                                1
Pork Sausage                                   1
Sausage And Bacon Products                     1
Chicken Biryani Frozen Entrées                 1
Cooked Country Hams                            1
Chicken Frankfurters                           1
Fully Cooked Chicken Breast Strips             1
Frozen Chicken And Sausage Gumbo               1
Chicken Breast Wrap Sandwiches                 1
Frozen Ground Beef Products                    1
Chicken Wrap Sandwiches                        1
Polish Sausage                                 1
Chicken Salad Products                         1
RTE Beef Products   

## Before doing this, work on those dtype for the columns!

In [29]:
rcl_rsn_dict = {}

def get_rcl_rsn_vc(df_dict):
    for year, df in df_dict.items():
        key = 'rcl_rsn_' + year
        df = pd.DataFrame(df['Recall Reason'].value_counts())
        df = df.transpose()
        df = df.rename({'Recall Reason': year})
        rcl_rsn_dict[key] = df

In [30]:
get_rcl_rsn_vc(df_dict)

In [31]:
vc_dfs = list(rcl_rsn_dict.values())

In [32]:
recall_reasons_combined = pd.concat(vc_dfs, sort=False)

In [33]:
recall_reasons_combined

,Listeria monocytogenes,Undeclared Allergen,E. coli O157:H7,Other,Processing Defect,Misbranding,Chemical Contamination,Extraneous Material,Salmonella,Residue,Undeclared Substance,E.coli O157:H7,Unapproved Substance,E. coli O26*,"E. coli O157:H7, E. coli O145, E. coli O45",E. coli O121,"E. coli O103, O111, O121, O145, O26, O45",E. coli
2005,30,9,5.0,3,2.0,2.0,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,6,9,8.0,3,2.0,NaN,NaN,4,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,11,12,22.0,3,5.0,NaN,NaN,2,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,15,7,17.0,7,NaN,NaN,3.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,8,13,NaN,13,3.0,2.0,NaN,5,6.0,NaN,1.0,16.0,2.0,NaN,NaN,NaN,NaN,NaN
2010,8,18,11.0,12,2.0,2.0,NaN,7,7.0,2.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2011,11,40,NaN,13,2.0,NaN,NaN,5,10.0,1.0,7.0,13.0,1.0,NaN,NaN,NaN,NaN,NaN
2012,16,29,5.0,9,1.0,NaN,NaN,13,2.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,9,25,7.0,14,2.0,NaN,NaN,10,4.0,NaN,2.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN
2014,7,43,4.0,23,4.0,NaN,NaN,6,4.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [34]:
recall_reasons_combined = recall_reasons_combined.fillna(0)

In [35]:
recall_reasons_combined = recall_reasons_combined.astype('int64')

In [36]:
recall_reasons_combined

,Listeria monocytogenes,Undeclared Allergen,E. coli O157:H7,Other,Processing Defect,Misbranding,Chemical Contamination,Extraneous Material,Salmonella,Residue,Undeclared Substance,E.coli O157:H7,Unapproved Substance,E. coli O26*,"E. coli O157:H7, E. coli O145, E. coli O45",E. coli O121,"E. coli O103, O111, O121, O145, O26, O45",E. coli
2005,30,9,5,3,2,2,1,1,0,0,0,0,0,0,0,0,0,0
2006,6,9,8,3,2,0,0,4,1,1,0,0,0,0,0,0,0,0
2007,11,12,22,3,5,0,0,2,1,0,2,0,0,0,0,0,0,0
2008,15,7,17,7,0,0,3,5,0,0,0,0,0,0,0,0,0,0
2009,8,13,0,13,3,2,0,5,6,0,1,16,2,0,0,0,0,0
2010,8,18,11,12,2,2,0,7,7,2,1,0,0,1,0,0,0,0
2011,11,40,0,13,2,0,0,5,10,1,7,13,1,0,0,0,0,0
2012,16,29,5,9,1,0,0,13,2,0,7,0,0,0,0,0,0,0
2013,9,25,7,14,2,0,0,10,4,0,2,0,0,0,1,1,0,0
2014,7,43,4,23,4,0,0,6,4,0,2,0,0,0,0,0,1,0


In [37]:
recall_2005['Recall Class'].value_counts()

NameError: name 'recall_2005' is not defined